In [113]:
import pandas as pd
import datetime as dt

#grab flight plan
flight_plan = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\iss_flight_plan_20220101-20251231.csv')

#query to only use dates where the event is dock and convert it to a list
dock_days_list = flight_plan[flight_plan['event'] == 'Dock']['datedim'].to_list()

#convert list of date strings into a list of datetime objects
dates_list = [dt.datetime.strptime(date, '%m/%d/%Y').date() for date in dock_days_list]

#print the days difference between each date
num_days = pd.Series(dates_list).diff().dt.days.iloc[1:].astype(int).tolist()
print("number of daays til next dock\n",num_days)




number of daays til next dock
 [4, 25, 22, 18, 24, 13, 43, 67, 15, 22, 12, 18, 76, 15, 5, 13, 67, 2, 13, 59, 21, 2, 19, 48, 29, 41, 10, 10, 17, 2, 15, 8, 30, 22, 7, 22, 29, 35, 16, 6, 14, 9, 44, 33, 4, 22, 1, 34, 3, 19, 17, 62, 4, 27, 48, 2, 29, 7, 25, 28, 25, 25]


In [114]:
#grab crew members per dock
crewF = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\iss_flight_plan_crew_20220101-20251321.csv')

crew_per_dock = []

for i, date in enumerate(dock_days_list):
    crewNum = crewF[crewF['datedim'] == date]['crew_count'].tolist()
    Amount = sum(crewNum)
    crew_per_dock.append(Amount)

#number of crew members on board during each dock
print("number of crew per docking\n",crew_per_dock)


number of crew per docking
 [7, 7, 10, 11, 11, 7, 7, 7, 10, 11, 7, 7, 7, 7, 7, 11, 7, 11, 11, 7, 7, 7, 11, 10, 7, 7, 11, 11, 7, 7, 11, 7, 10, 9, 7, 11, 7, 7, 11, 7, 7, 10, 10, 11, 7, 7, 7, 7, 7, 11, 7, 10, 11, 11, 7, 7, 11, 7, 10, 7, 7, 11, 7]


In [115]:
#grab rates
rater = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\rates_definition.csv')


#consumable must match a consumable on the rates csv. case sensitive.
consumable = 'Pretreat Tanks'
consumable = 'Air'
consumable = 'Water'


#sum of generated
sum_generated = sum(rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'generation')]['rate'].to_list())

#grabs the usage rates of consumable
usage_list = rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'usage')]['rate'].to_list()

#if units contains 'crew'
crew_check = rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'usage')]['units'].tolist()

sum_usage_list = []
sum_usage_crew_list = []

for i, j in enumerate(crew_check):
    if j.find('Crew') != -1:
        sum_usage_crew_list.append(usage_list[i])
    else:
        sum_usage_list.append(usage_list[i])



sum_usage = sum(sum_usage_list)
sum_usage_crew = sum(sum_usage_crew_list)

print(crew_check)
print(usage_list)
print('------------')
print(sum_usage_list)
print(sum_usage_crew_list)
print(sum_usage)
print(sum_usage_crew)





['Liters/Day', 'Liters/Crew/Day', 'Liters/Crew/Day', 'Liters/Day']
[1.0, 2.84, 2.5, 10.5]
------------
[1.0, 10.5]
[2.84, 2.5]
11.5
5.34


In [116]:
import math

#create a list of base rates 
listofrates = [sum_usage] * ((len(dock_days_list)) -1)
listofrates_crew = []
consumable_to_send = []
current_sum = 0
res_list = []

for i in range(0, len(listofrates)):
    listofrates_crew.append(crew_per_dock[i] * sum_usage_crew)

for i in range(0, len(listofrates)):
    res_list.append(((listofrates[i] + listofrates_crew[i]) - sum_generated) * num_days[i])


#rounding
for i, value in enumerate(res_list):
    current_sum+=value
    consumable_to_send.append(math.trunc(current_sum))
    current_sum = current_sum - math.trunc(current_sum)
print(consumable_to_send)
    


[101, 633, 910, 840, 1121, 329, 1089, 1697, 621, 1027, 304, 456, 1925, 380, 126, 607, 1697, 94, 607, 1494, 532, 51, 887, 1985, 734, 1039, 467, 467, 430, 51, 700, 203, 1240, 793, 177, 1027, 735, 886, 747, 152, 355, 372, 1819, 1541, 102, 557, 25, 861, 76, 887, 431, 2564, 187, 1260, 1216, 51, 1354, 177, 1034, 709, 633, 1167]


In [117]:
##RUN ALL or this wont work
dock_days_list.pop()

readable = pd.DataFrame(
    {'Docking Days': dock_days_list,
     'Amount of Consumable to Send': consumable_to_send
    })

display(readable)

,Docking Days,Amount of Consumable to Send
0,2/17/2022,101
1,2/21/2022,633
2,3/18/2022,910
3,4/9/2022,840
4,4/27/2022,1121
...,...,...
57,9/3/2025,177
58,9/10/2025,1034
59,10/5/2025,709
60,11/2/2025,633
